<a href="https://colab.research.google.com/github/PedrolyraC/electromechanical-conversion/blob/main/electromechanical_conversion_plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
def core_geometric_section(length: float, width: float)->float:
    return length*width

In [ ]:
def core_magnetic_section(length: float, width: float)->float:
    return 1.1*core_geometric_section(length, width)

In [39]:
def conductor_section(current, potency)->float:
    if potency == 500:
        return current/3
    if potency >= 500 and potency <= 1000:
        return current/2.5
    if potency >= 1000 and potency <= 3000:
        return current/2
    return 'Invalid value'

In [ ]:
def iron_core_weight(weight_per_cm: float, length_cable):
    return weight_per_cm*length_cable

In [40]:
def copper_core_weight(
    first_tension: float,
    second_tension: float,
    first_potency: float,
    second_potency: float,
    frequency: float,
    is_long_cable,
    is_two_primary_circuits: bool=False,
    is_two_secondary_circuits: bool=False
    )->float:
    geometric_section = calculate_magnectic_section_core(second_potency, frequency, is_long_cable, is_two_primary_circuits, is_two_secondary_circuits)
    aux = geometric_section
    a = round(math.sqrt(aux*1.1))
    b = geometric_section/a
    average_length = 2*a + 2*b + 0.5*a*math.pi
    s1 = conductor_section(first_tension, first_potency)
    s2 = conductor_section(second_tension, second_potency)
    n1 = first_tension*1.1
    n2 = second_tension*1.1
    section_copper = n1*s1 + n2*s2

    return (section_copper/100)*average_length*9

In [ ]:
def calculate_turns_number(
    tension:float,
    sm:float,
    frequency:float,
    bm:float = 11300
    )->float:

    return (tension/sm)*(1e8/(4.44*bm*frequency))

In [ ]:
def exec_possibility(
    first_tension: float,
    second_tension: float,
    first_depth: float,
    first_width: float,
    second_depth: float,
    second_width: float,
    frequency: float,
    bm:float = 11300
    )->bool:
    s1 = core_magnetic_section(first_depth, first_width)
    s2 = core_magnetic_section(second_depth, second_width)
    n1 = calculate_turns_number(first_tension, s1, frequency, bm)
    n2 = calculate_turns_number(second_tension, s2, frequency, bm)
    section_copper = n1*s1 + n2*s2
    section_window = 0.5

    if (section_window/section_copper) < 3:
        print('Impossible to project thistransformer. Please, reproject the transformer with a bigger core')
        return False
    return True

In [31]:
def calculate_magnectic_section_core(
    potency: float,
    frequency: float,
    is_long_cable: bool,
    is_two_primary_circuits: bool=False,
    is_two_secondary_circuits: bool=False
    )->float:
    standard_cables = 7.5*(math.sqrt(potency/frequency))
    long_cables = 6*(math.sqrt(potency/frequency))

    if is_two_primary_circuits is False and is_two_secondary_circuits is False:
        if is_long_cable:
            return long_cables
        return standard_cables

    if is_two_primary_circuits is True and is_two_secondary_circuits is False:
        standard_cables = 7.5*(math.sqrt(1.25*potency/frequency))
        long_cables = 6*(math.sqrt(1.25*potency/frequency))

        if is_long_cable:
            return long_cables
        return standard_cables

    if is_two_primary_circuits is True and is_two_secondary_circuits is True:
        standard_cables = 7.5*(math.sqrt(1.5*potency/frequency))
        long_cables = 6*(math.sqrt(1.5*potency/frequency))

        if is_long_cable:
            return long_cables
        return standard_cables

    print('Invalid option')
    return [0, 0]

In [ ]:
def calculate_magnetic_curve(sheet):
